In [ ]:
try:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh" -O "/tmp/firedrake-install.sh"
    !bash "/tmp/firedrake-install.sh"
    from firedrake import *  # noqa: F401
except:
    from firedrake import *  # noqa: F401

# Stokes Eigenvalue Problem (Streamfunction Formulation)

Solving the Stokes eigenvalue problem on a unit square domain using a streamfunction formulation $\psi$ where $u = \nabla \times \psi$.
Using a $C^0$ Interior Penalty method with CG degree 2 elements for the biharmonic operator.

In [ ]:
# 1. Mesh Definition
# Unit square mesh with N x N elements
N = 16
mesh = UnitSquareMesh(N, N)

In [ ]:
# 2. Function Spaces
# Streamfunction: CG (Continuous Galerkin) degree 2
# We use a C0 Interior Penalty method for the biharmonic operator.
V = FunctionSpace(mesh, "CG", 2)

# 3. Trial and Test Functions
psi = TrialFunction(V)
phi = TestFunction(V)

u = curl(psi)
v = curl(phi)

In [ ]:
# 5. Problem Parameters
nu = Constant(0.01)  # Kinematic viscosity
# No forcing term for eigenvalue problem

In [ ]:
# 6. Variational Form
# Biharmonic operator with C0 Interior Penalty method
# A(psi, phi) = (nu * Delta psi, Delta phi) + penalty terms
# M(psi, phi) = (grad psi, grad phi) -> This corresponds to -Delta psi on RHS

n = FacetNormal(mesh)
h = CellDiameter(mesh)
sigma = Constant(20.0)  # Penalty parameter

def a_biharmonic(u, v):
    
    dev = lambda u_ref : sym(grad(u_ref))
    jump_dev = lambda u_ref : 2 * avg(sym(outer(u_ref, n)))

    # Volume term: 2 * dev(u) : dev(v)
    term = 2 * inner(dev(u), dev(v)) * dx
    
    # Interior Facets
    term -= 2 * inner(avg(dev(u)), jump_dev(v)) * dS
    term -= 2 * inner(avg(dev(v)), jump_dev(u)) * dS
    term += 2 * (sigma / avg(h)) * inner(jump_dev(u), jump_dev(v)) * dS
    
    # # Boundary Facets (Weak imposition of no-slip BC u=0)
    # term -= inner(dev(u), outer(v, n)) * ds
    # term -= inner(dev(v), outer(u, n)) * ds
    # term += (sigma / h) * inner(u, v) * ds
    
    return term

A = a_biharmonic(u, v)
M = inner(u, v) * dx

In [ ]:
# 7. Solver
# Boundary conditions: psi = 0 on boundary (u.n = 0)
bcs = [DirichletBC(V, Constant(0.0), "on_boundary")]

eigenproblem = LinearEigenproblem(A, M, bcs=bcs)

# Request 6 eigenvalues
n_evals = 6
solver = LinearEigensolver(eigenproblem, n_evals, solver_parameters={
    "eps_type": "krylovschur",
    "eps_target": 0.0,
    "eps_which": "TARGET_MAGNITUDE",
    "st_type": "sinvert",
    "st_pc_type": "lu",
    "st_pc_factor_mat_solver_type": "mumps"
})

n_found = solver.solve()
print(f"Found {n_found} eigenvalues.")

In [ ]:
# 8. Output
for i in range(n_found):
    lam = solver.eigenvalue(i)
    print(f"Eigenvalue {i}: {lam}")

# # Save the first eigenmode
# # The solver returns the streamfunction psi
# psi_mode_real, psi_mode_imag = solver.eigenfunction(0)
# psi_mode_real.rename("Streamfunction")

# # Compute velocity u = curl(psi)
# # In 2D, curl(psi) = (d(psi)/dy, -d(psi)/dx)
# u_mode = project(curl(psi_mode_real), VectorFunctionSpace(mesh, "CG", 1))
# u_mode.rename("Velocity")

# outfile = File("stokes_streamfunction.pvd")
# outfile.write(psi_mode_real, u_mode)
# print("Saved first eigenmode to stokes_streamfunction.pvd")